<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Introduction

In this notebook, we will convert addresses into their equivalent latitude and longitude values. Also, we will use the Foursquare API to explore neighborhoods in Toronto. We will use the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. We will use the *k*-means clustering algorithm to complete this task. Finally, we will use the Folium library to visualize the neighborhoods in Toronto and their emerging clusters.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in Toronto</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         237 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0        conda-forge
    geopy:         1.20.0-py_0      conda-forge

The following pac

<a id='item1'></a>

## 1. Download and Explore Dataset

I have downloaded the dataset and placed it on the server, we can simply use a DataFrame to read the file

In [4]:
df = pd.read_csv('Toronto.csv')

Let's inspect the data

In [5]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
df.shape

(288, 3)

Remove rows were Borough = 'Not assigned'

In [7]:
df = df[df.Borough != 'Not assigned']

In [8]:
df.shape

(211, 3)

Check If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

In [9]:
df[df.Neighbourhood == 'Not assigned'] 

,Postcode,Borough,Neighbourhood
8,M7A,Queen's Park,Not assigned


In [10]:
df.loc[8].Neighbourhood = df.loc[8].Borough

In [11]:
df[df.Neighbourhood == 'Not assigned'] 

,Postcode,Borough,Neighbourhood


Identify postcodes with more than one city

In [12]:
df.groupby(['Postcode','Borough']).count()

,,Neighbourhood
Postcode,Borough,
M1B,Scarborough,2
M1C,Scarborough,3
M1E,Scarborough,3
M1G,Scarborough,1
M1H,Scarborough,1
M1J,Scarborough,1
M1K,Scarborough,3
M1L,Scarborough,3
M1M,Scarborough,3


Group over Postcode and Borough and aggregate the Neighborhoods

In [13]:
toronto_postal = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [14]:
toronto_postal.head(200) #show entire dataframe

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


Here is the shape of the Toronto Postal DataFrame

In [15]:
toronto_postal.shape

(103, 3)

Load Geos

In [16]:
df = pd.read_csv('Geospatial_Coordinates.csv')

In [ ]:
Show sample records

In [27]:
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge the dataframes

In [29]:
toronto_data = toronto_postal.merge(df, left_on='Postcode', right_on='Postal Code')

In [30]:
toronto_data.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


Drop the 'Postal Code' column

In [39]:
neighborhoods = toronto_data.drop(columns=['Postal Code'])

In [40]:
neighborhoods.shape

(103, 5)

Take a look at the empty dataframe to confirm that the columns are as intended.

Then let's loop through the data and fill the dataframe one row at a time.

Quickly examine the resulting dataframe.

In [41]:
neighborhoods.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [42]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Toronto.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [43]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [47]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in York. So let's slice the original dataframe and create a new dataframe of the Manhattan data.

In [48]:
york_data = neighborhoods[neighborhoods['Borough'] == 'York'].reset_index(drop=True)
york_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M6C,York,Humewood-Cedarvale,43.693781,-79.428191
1,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512
2,M6M,York,"Del Ray, Keelesdale, Mount Dennis, Silverthorn",43.691116,-79.476013
3,M6N,York,"The Junction North, Runnymede",43.673185,-79.487262
4,M9N,York,Weston,43.706876,-79.518188


Let's get the geographical coordinates of Manhattan.

In [49]:
address = 'York, Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of York, Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of York, Toronto, Ontario are 43.6896191, -79.479188.


As we did with all of New York City, let's visualizat Manhattan the neighborhoods in it.

In [51]:
# create map of Manhattan using latitude and longitude values
map_york = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(york_data['Latitude'], york_data['Longitude'], york_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_york)  
    
map_york

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [98]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: <hidden>')
print('CLIENT_SECRET: <hidden>' )

Your credentails:
CLIENT_ID: <hidden>
CLIENT_SECRET: <hidden>


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [54]:
york_data.loc[0, 'Neighbourhood']

'Humewood-Cedarvale'

Get the neighborhood's latitude and longitude values.

In [57]:
neighbourhood_latitude = york_data.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = york_data.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = york_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Humewood-Cedarvale are 43.6937813, -79.42819140000002.


#### Now, let's get the top 100 venues that are in Humewood-Cedarvale within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [58]:
# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL




'https://api.foursquare.com/v2/venues/explore?&client_id=FD0EOCRMLZBUH0QSGNF0NSNXMPAMT2I5OLGUSGQSAVMDCDZH&client_secret=QXC0KBN1WHQ0I40KKACAPQRC2OS1EBKOWCE0LWBSUJD1IDMS&v=20180605&ll=43.6937813,-79.42819140000002&radius=500&limit=100'

Double-click __here__ for the solution.
<!-- The correct answer is:
LIMIT = 100 # limit of number of venues returned by Foursquare API
-->

<!--
radius = 500 # define radius
-->

<!--
\\ # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
--> 

Send the GET request and examine the resutls

In [59]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d897fde02a17200212c0ef9'},
 'response': {'headerLocation': 'Cedarvale',
  'headerFullLocation': 'Cedarvale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.6982813045, 'lng': -79.4219793104081},
   'sw': {'lat': 43.689281295499995, 'lng': -79.43440348959193}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b0afc19f964a520212b23e3',
       'name': 'Cedarvale Park',
       'location': {'address': '100 glen cedar',
        'crossStreet': 'Strathearn',
        'lat': 43.692534923091934,
        'lng': -79.42870527613704,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.692534923091934,
          'lng': -79.42870527613704}],
        'distance': 144,


From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [60]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [61]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Cedarvale Park,Field,43.692535,-79.428705
1,Glen Cedar Park,Playground,43.695399,-79.429253
2,Phil White Arena,Hockey Arena,43.691303,-79.431761
3,Cedarvale Ravine,Trail,43.690188,-79.426106


And how many venues were returned by Foursquare?

In [62]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


<a id='item2'></a>

## 2. Explore Neighborhoods in York

#### Let's create a function to repeat the same process to all the neighborhoods in York

In [63]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [64]:
# type your answer here

york_venues = getNearbyVenues(names=york_data['Neighbourhood'],
                                   latitudes=york_data['Latitude'],
                                   longitudes=york_data['Longitude']
                                  )



Humewood-Cedarvale
Caledonia-Fairbanks
Del Ray, Keelesdale, Mount Dennis, Silverthorn
The Junction North, Runnymede
Weston


Double-click __here__ for the solution.
<!-- The correct answer is:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )
--> 

#### Let's check the size of the resulting dataframe

In [65]:
print(york_venues.shape)
york_venues.head()

(20, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Humewood-Cedarvale,43.693781,-79.428191,Cedarvale Park,43.692535,-79.428705,Field
1,Humewood-Cedarvale,43.693781,-79.428191,Glen Cedar Park,43.695399,-79.429253,Playground
2,Humewood-Cedarvale,43.693781,-79.428191,Phil White Arena,43.691303,-79.431761,Hockey Arena
3,Humewood-Cedarvale,43.693781,-79.428191,Cedarvale Ravine,43.690188,-79.426106,Trail
4,Caledonia-Fairbanks,43.689026,-79.453512,KFC,43.690647,-79.456326,Fast Food Restaurant


Let's check how many venues were returned for each neighborhood

In [66]:
york_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Caledonia-Fairbanks,5,5,5,5,5,5
"Del Ray, Keelesdale, Mount Dennis, Silverthorn",6,6,6,6,6,6
Humewood-Cedarvale,4,4,4,4,4,4
"The Junction North, Runnymede",3,3,3,3,3,3
Weston,2,2,2,2,2,2


#### Let's find out how many unique categories can be curated from all the returned venues

In [67]:
print('There are {} uniques categories.'.format(len(york_venues['Venue Category'].unique())))

There are 18 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [68]:
# one hot encoding
york_onehot = pd.get_dummies(york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
york_onehot['Neighbourhood'] = york_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [york_onehot.columns[-1]] + list(york_onehot.columns[:-1])
york_onehot = york_onehot[fixed_columns]

york_onehot.head()

,Neighbourhood,Bar,Bus Line,Caribbean Restaurant,Check Cashing Service,Coffee Shop,Convenience Store,Discount Store,Fast Food Restaurant,Field,Hockey Arena,Market,Park,Pizza Place,Playground,Restaurant,Sandwich Place,Trail,Women's Store
0,Humewood-Cedarvale,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,Humewood-Cedarvale,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,Humewood-Cedarvale,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,Humewood-Cedarvale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,Caledonia-Fairbanks,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [69]:
york_onehot.shape

(20, 19)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [70]:
york_grouped = york_onehot.groupby('Neighbourhood').mean().reset_index()
york_grouped

,Neighbourhood,Bar,Bus Line,Caribbean Restaurant,Check Cashing Service,Coffee Shop,Convenience Store,Discount Store,Fast Food Restaurant,Field,Hockey Arena,Market,Park,Pizza Place,Playground,Restaurant,Sandwich Place,Trail,Women's Store
0,Caledonia-Fairbanks,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.2,0.00,0.00,0.2,0.4,0.000000,0.00,0.000000,0.000000,0.00,0.2
1,"Del Ray, Keelesdale, Mount Dennis, Silverthorn",0.166667,0.000000,0.000000,0.166667,0.166667,0.0,0.166667,0.0,0.00,0.00,0.0,0.0,0.000000,0.00,0.166667,0.166667,0.00,0.0
2,Humewood-Cedarvale,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.25,0.25,0.0,0.0,0.000000,0.25,0.000000,0.000000,0.25,0.0
3,"The Junction North, Runnymede",0.000000,0.333333,0.333333,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.00,0.0,0.0,0.333333,0.00,0.000000,0.000000,0.00,0.0
4,Weston,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.0,0.00,0.00,0.0,0.5,0.000000,0.00,0.000000,0.000000,0.00,0.0


#### Let's confirm the new size

In [71]:
york_grouped.shape

(5, 19)

#### Let's print each neighborhood along with the top 5 most common venues

In [73]:
num_top_venues = 5

for hood in york_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = york_grouped[york_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Caledonia-Fairbanks----
                  venue  freq
0                  Park   0.4
1         Women's Store   0.2
2                Market   0.2
3  Fast Food Restaurant   0.2
4                 Trail   0.0


----Del Ray, Keelesdale, Mount Dennis, Silverthorn----
                   venue  freq
0                    Bar  0.17
1  Check Cashing Service  0.17
2            Coffee Shop  0.17
3         Sandwich Place  0.17
4         Discount Store  0.17


----Humewood-Cedarvale----
          venue  freq
0  Hockey Arena  0.25
1         Trail  0.25
2    Playground  0.25
3         Field  0.25
4        Market  0.00


----The Junction North, Runnymede----
                  venue  freq
0  Caribbean Restaurant  0.33
1           Pizza Place  0.33
2              Bus Line  0.33
3                   Bar  0.00
4                Market  0.00


----Weston----
               venue  freq
0  Convenience Store   0.5
1               Park   0.5
2                Bar   0.0
3             Market   0.0
4              T

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [74]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [86]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = york_grouped['Neighbourhood']

for ind in np.arange(york_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(york_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Caledonia-Fairbanks,Park,Women's Store,Fast Food Restaurant,Market,Bus Line,Caribbean Restaurant,Check Cashing Service,Coffee Shop,Convenience Store,Discount Store
1,"Del Ray, Keelesdale, Mount Dennis, Silverthorn",Bar,Sandwich Place,Restaurant,Check Cashing Service,Coffee Shop,Discount Store,Fast Food Restaurant,Bus Line,Caribbean Restaurant,Convenience Store
2,Humewood-Cedarvale,Field,Playground,Hockey Arena,Trail,Discount Store,Bus Line,Caribbean Restaurant,Check Cashing Service,Coffee Shop,Convenience Store
3,"The Junction North, Runnymede",Bus Line,Caribbean Restaurant,Pizza Place,Women's Store,Fast Food Restaurant,Check Cashing Service,Coffee Shop,Convenience Store,Discount Store,Field
4,Weston,Park,Convenience Store,Women's Store,Fast Food Restaurant,Bus Line,Caribbean Restaurant,Check Cashing Service,Coffee Shop,Discount Store,Field


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [87]:
# set number of clusters
kclusters = 5

york_grouped_clustering = york_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 4, 1, 2, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [88]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

york_merged = york_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
york_merged = york_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

york_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M6C,York,Humewood-Cedarvale,43.693781,-79.428191,1,Field,Playground,Hockey Arena,Trail,Discount Store,Bus Line,Caribbean Restaurant,Check Cashing Service,Coffee Shop,Convenience Store
1,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512,3,Park,Women's Store,Fast Food Restaurant,Market,Bus Line,Caribbean Restaurant,Check Cashing Service,Coffee Shop,Convenience Store,Discount Store
2,M6M,York,"Del Ray, Keelesdale, Mount Dennis, Silverthorn",43.691116,-79.476013,4,Bar,Sandwich Place,Restaurant,Check Cashing Service,Coffee Shop,Discount Store,Fast Food Restaurant,Bus Line,Caribbean Restaurant,Convenience Store
3,M6N,York,"The Junction North, Runnymede",43.673185,-79.487262,2,Bus Line,Caribbean Restaurant,Pizza Place,Women's Store,Fast Food Restaurant,Check Cashing Service,Coffee Shop,Convenience Store,Discount Store,Field
4,M9N,York,Weston,43.706876,-79.518188,0,Park,Convenience Store,Women's Store,Fast Food Restaurant,Bus Line,Caribbean Restaurant,Check Cashing Service,Coffee Shop,Discount Store,Field


Finally, let's visualize the resulting clusters

In [97]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(york_merged['Latitude'], york_merged['Longitude'], york_merged['Neighbourhood'], york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [91]:
york_merged.loc[york_merged['Cluster Labels'] == 0, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,York,0,Park,Convenience Store,Women's Store,Fast Food Restaurant,Bus Line,Caribbean Restaurant,Check Cashing Service,Coffee Shop,Discount Store,Field


#### Cluster 2

In [93]:
york_merged.loc[york_merged['Cluster Labels'] == 1, york_merged.columns[[1] + list(range(5,york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,York,1,Field,Playground,Hockey Arena,Trail,Discount Store,Bus Line,Caribbean Restaurant,Check Cashing Service,Coffee Shop,Convenience Store


#### Cluster 3

In [94]:
york_merged.loc[york_merged['Cluster Labels'] == 2, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,York,2,Bus Line,Caribbean Restaurant,Pizza Place,Women's Store,Fast Food Restaurant,Check Cashing Service,Coffee Shop,Convenience Store,Discount Store,Field


#### Cluster 4

In [95]:
york_merged.loc[york_merged['Cluster Labels'] == 3, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,York,3,Park,Women's Store,Fast Food Restaurant,Market,Bus Line,Caribbean Restaurant,Check Cashing Service,Coffee Shop,Convenience Store,Discount Store


#### Cluster 5

In [96]:
york_merged.loc[york_merged['Cluster Labels'] == 4, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,York,4,Bar,Sandwich Place,Restaurant,Check Cashing Service,Coffee Shop,Discount Store,Fast Food Restaurant,Bus Line,Caribbean Restaurant,Convenience Store
